In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/

In [ ]:
#Giải nén file trainSet.zip (nếu đã giải nén rồi thì bỏ qua bước này)
! unzip /content/drive/MyDrive/submit/testSet.zip -d /content/drive/MyDrive/submit/

Đảm bảo đã up file lên drive và giải nén, sửa tất cả đường dẫn. Nhớ bật colab GPU

In [ ]:
! pip install transformers decord ffmpeg

**Restart kernal để update các package, sau đó chạy tiếp, không cần chạy lại mấy thứ ở trên ...**

In [ ]:
import os
os._exit(00)

**Chương trình test 1 video để kiểm tra model và video có tồn tại không**

In [ ]:
import transformers
import decord
video_cls = transformers.pipeline(model="bluebird089/videomae-base-finetuned-kinetics-finetuned-ok-hai", device = 'cuda:0')
#Thư mục chứa video test (sửa đường dẫn nếu cần thiết)
location_video = "/content/drive/MyDrive/submit/testSet/leg raises/leg_raises_18.mp4"
video_cls(location_video)

Chuẩn hóa class, vì btc đặt tên thư mục và video khác tên nhau nên chúng tôi cần chuẩn hóa trước khi đưa ra kết quả
dbp -> decline bench press
tricep Pushdown -> tricep pushdown
...

In [ ]:
def changeName(label):
  if label == 'dbp':
    return 'decline bench press'
  elif label == 'tricep pushdown':
    return 'tricep Pushdown'
  elif label == 'pull up':
    return 'pull Up'
  else:
    return label

Hàm chuyển đổi sang H264 nếu gặp lỗi, sửa đường dẫn nếu muốn

In [ ]:
import subprocess
import os

def convert_video(input_path):
    parts = input_path.split('/')
   
    videoName = parts[-1]

    className = parts[-2]
    #Tạo thư mục chứa video đã chuyển đổi (sửa đường dẫn nếu cần thiết)
    output_dir = f"/content/drive/MyDrive/Output/{className}"
    if not os.path.exists(output_dir):
      os.makedirs(output_dir)
      print("Đã tạo")
    #Thư mục chứa video đã chuyển đổi (sửa đường dẫn nếu cần thiết)
    output_path = f"/content/drive/MyDrive/Output/{className}/{videoName}"
    command = [
        'ffmpeg',
        '-i', input_path,  
        '-c:v', 'libx264',  
        '-crf', '23',  
        output_path  
    ]

    subprocess.run(command)

    print(f'Video đã được chuyển đổi và lưu tại: {output_path}')

**Chương trình test 1 dataset, cần sửa data_dir lưu dataTest, cấu trúc giống thư mục BTC gửi**

In [ ]:
import os
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
import decord
import time 

video_cls = pipeline(model="bluebird089/videomae-base-finetuned-kinetics-finetuned-ok-hai", device="cuda:0")

# Đường dẫn dataTest (sửa đường dẫn nếu cần thiết)
data_dir = "/content/drive/MyDrive/submit/testSet"

y_true = []
y_pred = []

for className in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, className)
        if os.path.isdir(class_dir):
            for video_file in os.listdir(class_dir):
              try:
                video_path = os.path.join(class_dir, video_file)
                print(video_path)
                prediction = video_cls(video_path)
                predicted_class = prediction[0]['label']
                predicted_class = changeName(predicted_class)
                y_true.append(className)
                y_pred.append(predicted_class)
                print(className)
                print(predicted_class, className == predicted_class)

              except Exception as e:
                print("Lỗi do định dạng video, đang chuyển sang định dạng H264")
                convert_video(video_path)

Chạy các video đã chuyển sang H264 và tính confusion matrix, F1.
Nếu bạn có sửa đường dẫn lưu file H264, thì cần sửa dưới này

In [ ]:
import os
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
import decord
import time 

# Load model (sửa đường dẫn nếu cần thiết)
data_dir = "/content/drive/MyDrive/Output"


for className in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, className)
        if os.path.isdir(class_dir):
            for video_file in os.listdir(class_dir):
              try:
                video_path = os.path.join(class_dir, video_file)
                print(video_path)
                prediction = video_cls(video_path)
                predicted_class = prediction[0]['label']
                predicted_class = changeName(predicted_class)
                print(className)
                y_true.append(className)
                y_pred.append(predicted_class)
                print(predicted_class, className == predicted_class)

                time.sleep(0.2) 
              except Exception as e:
                print(e)

# Calculate confusion matrix and F1 score
conf_matrix = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='micro')

# Print or plot metrics
print("F1 Score:", f1)
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
